# 🐙 Tutoriel pour l'utilisation de **Kami** (Kraken Model Inspector) 
<br>

Ce tutoriel a été élaboré par **Lucas Terriel** (Inria - ALMAnaCH) et **Alix Chagué** (Inria - ALMAnaCH) et publié sur le dépôt du projet [Kami](https://gitlab.inria.fr/dh-projects/kami/kami-lib)  ([pour citer le projet](https://gitlab.inria.fr/dh-projects/kami/kami-lib/-/blob/master/CITATION.CFF)).

Nous en avons simplifié l'écriture pour une utilisation exclusivement locale (le [tutoriel original](https://cutt.ly/WT3Ahx1) est publié sur Google Collab).

Date de la version utilisée : 24/08/2021.


## Introduction

Ce notebook vise à présenter l'utilisation du *package* Python **Kami** à travers l'exemple d'une chaîne de traitement complète : import des données => résultats du modèle HTR/OCR => export des résultats en CSV 



## Étape préliminaire : Installation du *package* Python Kami
🚀 Lancer la cellule et attendre la fin de l'exécution. (Cela peut prendre un certain temps)

⚠️ **N.B.** : *Ignorer les erreurs relatives à l'installation des packages scikit-learn, torch et torchtext, à la fin de l'éxecution de la cellule. (erreurs inhérentes à google colab, relancer la cellule les erreurs disparaissent)*

In [ ]:
# vérification de la version Python utilisé 
!python --version
# installation des packages nécéssaires
!pip install kamilib
#!pip install -i https://test.pypi.org/simple/ kamilib
# lister vos dépendances en dé-commentant la ligne qui suit
!pip freeze

## Étape 1 : Importer les *packages* Python pour la chaîne de traitement

🚀 Lancer la cellule.

⚠️ **N.B** : *Ignorer les warnings de dépréciations ... (relancer la cellule pour les faire disparaître)*

In [ ]:
import csv
from datetime import datetime
import io
import pprint

import pandas as pd

from kami.Kami import Kami

## Étape 2 : Tester le *package* Kami

🚀 Lancer la cellule.

✔️ **N.B** : *Si le code s'exécute avec succès, vous pouvez passer à la suite du notebook.*

In [ ]:
# deux phrases d'exemples
ground_truth = "1: J'aime Python comme langage de programmation !!"
hypothesis = "2: J'adore python comme langage de développement web ?"

# Appel de la classe
k = Kami([ground_truth, hypothesis], verbosity=False, truncate=True, percent=True, round_digits='0.01')

# Affichage des phrases d'exemples
reference = k.reference
prediction = k.prediction


print("--------------------")
print(reference)
print("--------------------")
print(prediction)
print("--------------------")

pprint.pprint(k.scores.board)

## Étape 3 : Importer les données (image, vérité terrain, et modèle HTR/OCR Kraken)

⚠️ Attention, si vous redémaré le noyau du notebook, vos données disparaissent, il est nécéssaire de les importer à nouveau.


On indiquer le chemin vers les données :

In [ ]:
# ground_truth est un chemin vers un fichier de test (éviter un fichier d'entraînement)
ground_truth = "../entrainements/veriteTerrain/CdS02_Konv019_0005.xml"
# image est le chemin vers l'image associée au précédent fichier
image = "../entrainements/veriteTerrain/CdS02_Konv019_0005.jpg"

model = "../entrainements/modeles-rec/souvay.mlmodel"

## Étape 4 : Configuration des paramètres d'évaluation (optionnelle)

Vous pouvez ignorer cette étape en utilisant les paramètres par défaut ci-dessous en éxécutant la cellule.

⚒️ Options disponibles
-------------------

* **transforms** : Permet d'appliquer des transformations au texte (*pre-processing*) afin de faire varier les scores en sortie, vous pouvez combiner ces options ex. transforms="DLP" (attention l'ordre peut avoir un impact). 
  - **D** : Supprime l'ensemble des chiffres et nombres;
  - **U** : Majusculise le texte;
  - **L** : Minusculise le texte;
  - **P** : Supprime la ponctuation;
  - **X** : Supprime les signes diacritiques;

* **verbosity** : Permet de récupérer des logs d'éxécution pour suivre le processus.

* **truncate** : Option à utiliser pour tronquer le résultat final.

* **percent** : Option pour afficher un résultat en pourcentage.

* **round_digits** : Option pour régler les chiffres après la virgule.

In [ ]:
transforms="XP"
verbosity=False
truncate=True
percent=True
round_digits='0.01'

## Étape 5 : Création de l'objet Kami pour opérer les scores 


🚀 Lancer la cellule. En cas d'erreur vérifier bien que vous avez lancer la cellule précédente.

Si vous utilisez du XML en vérité terrain, l'erreur : `Recognizers with segmentation types set() will be applied to segmentation of type baselines. This will likely result in severely degraded performace` apparait. Vous pouvez ignorer ces avertissements et passer à la suite.

In [ ]:
kevaluator = Kami(ground_truth,  
         model=model, 
         image=image,  
         apply_transforms=transforms,
         verbosity=verbosity, 
         truncate=truncate, 
         percent=True,  
         round_digits=round_digits) 

## Étape 6 : Visualiser les formes de textes (optionnelle)

🚀 Lancer la cellule pour visualiser le texte de vérité terrain et le texte prédit, avec ou sans transformations selon les options précédémment choisies.

⚠️ Il se peut que la taille de votre texte soit trop grande dans ce cas sélectionner des parties du texte via l'index pour afficher des extraits.

In [ ]:
print("GROUND TRUTH TEXT : \n")
print(kevaluator.reference)

# via l'index (en cas de texte trop long) :
# print(kevaluator.reference[0:50])

print(f"\n{'-' * 20}\n")

print("PREDICTION TEXT : \n")
print(kevaluator.prediction)

print(f"\n{'=' * 20}\n")

print("GROUND TRUTH TEXT (with transforms) : \n")
print(kevaluator.reference_preprocess)

print(f"\n{'*' * 20}\n")

print("PREDICTION TEXT (with transforms) : \n")
print(kevaluator.prediction_preprocess)

print(f"\n{'*' * 20}\n")

## Étape 7 : Visualiser les résultats obtenus dans un tableau


🚀 Lancer la cellule pour afficher les résultats.

Précisions sur les métriques 
----------------------------

Opérations (utilisés pour la distance) : 

* **hits** : nombre de caractères similaires entre la vérité terrain et la prédiction; 
* **substitutions** : nombre de substitutions opérées entre la vérité terrain et la prédiction; 	
* **deletions** : nombre de suppressions opérées entre la vérité terrain et la prédiction; 	
* **insertions** : nombre d'insertions opérées entre la vérité terrain et la prédiction; 	

Distance d'édition :

* **levensthein_distance_char** : distance de Levenshtein calculée sur les caractères;
* **levensthein_distance_words** : distance de Levenshtein calculée sur les mots;
* **hamming_distance** : distance de Hamming (Ø si les chaînes ne font pas exactement la même longueur, nombre si les chaînes font la même longueur mais que certains caractères divergent).

Scores généraux HTR/OCR :

* **wer** : *word error rate* ou taux d'erreur par mots. Le $WER$ est calculé sur la base des mots de la vérité terrain par rapport à la prédiction réalisée par le modèle. Il est généralement compris entre $[0, 1.0]$, plus il est proche de 0 plus la reconnaissance est bonne. En revanche, il n'est pas borné : une mauvaise reconnaissance peut entrainer un $WER > 1.0$. 
Il est calculé
\begin{align}
\frac{S + D + I}{N}
\end{align}
où $S$ est le nombre de substitions de mots, $D$ le nombre de mots surpprimés, et $I$ le nombre de mots insérés; $N$ correspond au nombre total de mots dans la chaîne de référence; cela correspond également à
\begin{align}
\frac{distance \, de \, Levenshtein \, sur \, les \, mots}{N}
\end{align}

* **cer** : *character error rate* ou taux d'erreur par caractères. Le $CER$ est calculé sur la base des caractères de la vérité terrain par rapport à la prédiction réalisée par le modèle. Il est généralement compris entre $[0, 1.0]$, plus il est proche de 0 plus la reconnaissance est bonne. En revanche, il n'est pas borné : une mauvaise reconnaissance peut entrainer un $CER > 1.0$.
Il est calculé
\begin{align}
\frac{S + D + I}{N}
\end{align}
où $S$ est le nombre de substitions de caractères, $D$ le nombre de caractères surpprimés, et $I$ le nombre de caractères insérés; $N$ correspond au nombre total de caractères dans la chaîne de référence; cela correspond également à
\begin{align}
\frac{distance \, de \, Levenshtein \, sur \, les \, caractères}{N}
\end{align}

* **wacc** 	: *word accuracy* ou taux de reconnaissance par mots calculé 
\begin{align}
1- WER 
\end{align}
En cas de très mauvaise reconnaissance ce taux peut-être négatif.

Scores de précision (emprunté à l'évaluation de la reconnaissance de la parole - système ASR) : 

* **mer** : *match error rate*
* **cil** : *character information lost*
* **cip** : *charcter information preserve*

 Voir :

 - [Morris, A. et al. “From WER and RIL to MER and WIL: improved evaluation measures for connected speech recognition.” INTERSPEECH (2004).](https://www.semanticscholar.org/paper/From-WER-and-RIL-to-MER-and-WIL%3A-improved-measures-Morris-Maier/8516531ff3bd874b66b811f0bd4e21a2d6b10e54?p2df)

 - [Errattahi, R. et al. "Automatic Speech Recognition Errors Detection and Correction: A Review". Procedia Computer Science. (2018).](https://www.researchgate.net/publication/324427003_Automatic_Speech_Recognition_Errors_Detection_and_Correction_A_Review)





In [ ]:
metadatas = {}
metrics = {}

now = datetime.now()
metadatas['DATETIME'] = now.strftime("%d_%m_%Y_%H:%M:%S")
metadatas['IMAGE'] = image
metadatas['GROUND_TRUTH'] = ground_truth
metadatas['MODEL'] = model

for key, value in kevaluator.scores.board.items():
  if type(value) != dict and key not in ['levensthein_distance_char', 
                                           'levensthein_distance_words',
                                           'hamming_distance', 
                                           'wer', 
                                           'cer', 
                                           'wacc', 
                                           'mer', 
                                           'cil', 
                                           'cip', 
                                           'hits', 
                                           'substitutions', 
                                           'deletions', 
                                           'insertions']:
    metadatas[key] = value
  else:
    metrics[key] = value

print(f"************* EVALUATION HTR/OCR REPORT FROM KAMI - {metadatas['DATETIME']} / MODEL : {metadatas['MODEL']} *************\n")

for key, value in metadatas.items():
  print(f"- {key} : {value}")

print("\n")

try:
  df_metrics = pd.DataFrame.from_dict(metrics)
except:
  df_metrics = pd.DataFrame.from_dict(metrics, orient='index')

df_metrics

## Étape 8 : Exporter les résultats en CSV

🚀 Lancer la cellule.

In [ ]:
name_csv = f"evaluation_report_kami_{metadatas['DATETIME']}_{metadatas['MODEL']}.csv"

with open(name_csv, 'w')  as csv_file:
  writer = csv.writer(csv_file, 
                      delimiter=',',
                      quotechar='|', 
                      quoting=csv.QUOTE_MINIMAL)
  for key, value in metadatas.items():
    row = []
    row.append(key)
    row.append(value)
    writer.writerow(row)

df_metrics.to_csv(name_csv, mode='a', header=True)


files.download(name_csv)